In [53]:
%pip install langchain langchain-community langchain_groq chromadb gradio

In [54]:
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_groq import ChatGroq

In [55]:
anime  = pd.read_csv("/anime_processed.csv")
anime.head()

,Name,Score,Genres,English name,Japanese name,Synopsis,Type,Rating,Episodes
0,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,"In the year 2071, humanity has colonized sever...",TV,R - 17+ (violence & profanity),26
1,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,"other day, another bounty—such is the life of ...",Movie,R - 17+ (violence & profanity),1
2,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,"Vash the Stampede is the man with a $$60,000,0...",TV,PG-13 - Teens 13 or older,26
3,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),ches are individuals with special powers like ...,TV,PG-13 - Teens 13 or older,26
4,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,It is the dark century and the people are suff...,TV,PG - Children,52


In [56]:
anime['combined_df'] =anime.apply(lambda row: f"Title: {row['Name']}. English Name : {row['English name']}. Japanese Name : {row['Japanese name']}. Overview : {row['Synopsis']}. Episodes : {row['Episodes']}. Score : {row['Score']}. Type : {row['Type']}. Rating : {row['Rating']}", axis = 1)

In [57]:
anime['combined_df'][0]

'Title: Cowboy Bebop. English Name : Cowboy Bebop. Japanese Name : カウボーイビバップ. Overview : In the year 2071, humanity has colonized several of the planets and moons of the solar system leaving the now uninhabitable surface of planet Earth behind. The Inter Solar System Police attempts to keep peace in the galaxy, aided in part by outlaw bounty hunters, referred to as "Cowboys." The ragtag team aboard the spaceship Bebop are two such individuals. Mellow and carefree Spike Spiegel is balanced by his boisterous, pragmatic partner Jet Black as the pair makes a living chasing bounties and collecting rewards. Thrown off course by the addition of new members that they meet in their travels—Ein, a genetically engineered, highly intelligent Welsh Corgi; femme fatale Faye Valentine, an enigmatic trickster with memory loss; and the strange computer whiz kid Edward Wong—the crew embarks on thrilling adventures that unravel each member\'s dark and mysterious past little by little. Well-balanced with 

In [58]:
anime[['combined_df']].to_csv('anime_updated.csv', index = False)

In [59]:
pd.read_csv("anime_updated.csv")

,combined_df
0,Title: Cowboy Bebop. English Name : Cowboy Beb...
1,Title: Cowboy Bebop: Tengoku no Tobira. Englis...
2,Title: Trigun. English Name : Trigun. Japanese...
3,Title: Witch Hunter Robin. English Name : Witc...
4,Title: Bouken Ou Beet. English Name : Beet the...
...,...
12693,Title: Heikousen. English Name : -. Japanese N...
12694,Title: Initial D Battle Stage 3. English Name ...
12695,Title: Nu Wushen de Canzhuo Spring Festival Sp...
12696,Title: Yakusoku no Neverland 2nd Season: Michi...


**Data** Loader and VectorStore using **Langchain**

In [60]:
loader = CSVLoader(file_path="/content/anime_updated.csv")
data = loader.load()

text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
texts = text_splitter.split_documents(data)
embeddings = HuggingFaceEmbeddings()
llm = ChatGroq(api_key = "gsk_NHgqXXXx3sxGksbu5u3aWGdyb3FYXj8BKec5Sk29Jn2pkD83RAtb", model = "llama-3.3-70b-versatile", temperature= 0 )

docssearch = Chroma.from_documents(texts, embeddings)

<ipython-input-60-3738cf599ba3>:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


**Querying Vector DB Store for movie recommendation**

In [61]:
query = "I am looking for an animated sci-fi movie. What could you suggest to me"
docs = docssearch.similarity_search(query, k =1)

In [62]:
docs

[Document(metadata={'source': '/content/anime_updated.csv', 'row': 6409}, page_content='combined_df: Title: SF Saiyuuki Starzinger (Movie). Overview : Princess Aurora, the only survivor of her planet destroyed by evil mutants, is resolved to put an end to the mutant menace in the galaxy along with three enhanced human cyborgs Jesse Dart, Porkos and Aramus, she travels space in the Cosmos Queen to the Dekkos System to fight the final battle. (Source: ANN) Genres : Adventure, Mecha, Sci-Fi, Shounen')]

**Using QA Retrieval for movie recommendation**

In [63]:
qa = RetrievalQA.from_chain_type(llm,
            chain_type = "stuff",
            retriever = docssearch.as_retriever(),
            return_source_documents = True)

In [64]:
query = "Can you recommend a romance anime with a unique storyline?"
result = qa({"query": query})

In [65]:
result['result']

'Based on the information I have, I would recommend "Marmalade Boy" for its unique storyline. The anime revolves around a high school girl whose parents get divorced and then swap partners with another couple, leading to a complex and interesting family dynamic. The story explores the romance that develops between the girl, Miki, and her new stepbrother, Yuu, amidst the challenges and complications that come with their new family situation. This setup offers a fresh and intriguing take on the traditional romance genre.'

In [66]:
result['source_documents'][0]

Document(metadata={'source': '/content/anime_updated.csv', 'row': 8249}, page_content="combined_df: Title: Saenai Heroine no Sodatekata: Ai to Seishun no Service-kai. Overview : Tomoya Aki, Megumi Katou, Eriri Spencer Sawamura, Utaha Kasumigaoka and Tomoya's cousin Michiru Hyoudou (all members of a doujin circle called Blessing Software) go out location scouting for their project, a video game. As Eriri attempts to get Megumi to act out a scenario for the main heroine, Utaha lures Tomoya away and forces him to act out a romantic scenario with her. (Source: Wikipedia) Genres : Harem, Comedy, Romance, Ecchi")

**Prompt Enginnering**

In [67]:
from langchain.prompts import PromptTemplate

template = """Anda adalah sistem pemberi rekomendasi film yang membantu pengguna menemukan anime yang sesuai dengan preferensi mereka.
Gunakan potongan konteks berikut untuk menjawab pertanyaan di akhir. Untuk setiap pertanyaan, sarankan tiga anime,
dengan deskripsi singkat tentang plot dan alasan mengapa pengguna mungkin menyukainya.
Jika Anda tidak tahu jawabannya, katakan saja Anda tidak tahu, jangan mencoba mengarang jawaban.

{context}

Question: {question}
Your response:"""

PROMPT  =  PromptTemplate(
    template=template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm,
            chain_type = "stuff",
            retriever = docssearch.as_retriever(),
            return_source_documents = True,
                                 chain_type_kwargs = chain_type_kwargs)

query = "Give me a list of comedy animes that are lighthearted and fun."
result = qa({"query": query})
print(result['result'])


Berikut adalah tiga rekomendasi anime komedi yang ringan dan menyenangkan:

1. **Shiawase no Katachi**: Anime ini merupakan komedi yang diadaptasi dari manga dengan judul yang sama. Dengan genre komedi, anime ini menjanjikan pengalaman menonton yang ringan dan menyenangkan.
2. **Eguchi Hisashi no Kotobuki Gorou Show**: Anime ini juga merupakan komedi yang diadaptasi dari manga karya Hisashi Eguchi. Dengan sentuhan parodi dan komedi, anime ini dapat menjadi pilihan yang tepat untuk mereka yang mencari tontonan ringan dan lucu.
3. **Kaminari Boy Pikkari★bee**: Anime ini merupakan komedi tentang seorang anak laki-laki yang suka bermain dan memiliki dua cerita dalam setiap episodenya. Dengan genre komedi dan petualangan, anime ini dapat menjadi pilihan yang menyenangkan untuk ditonton.

Semua anime ini memiliki elemen komedi yang kuat dan dapat menjadi pilihan yang tepat untuk mereka yang mencari tontonan ringan dan menyenangkan.


**Second Template**

In [68]:
from langchain.prompts import PromptTemplate

template_prefix = """Anda adalah sistem pemberi rekomendasi film yang membantu pengguna menemukan anime yang sesuai dengan preferensi mereka.
Gunakan potongan konteks berikut untuk menjawab pertanyaan di akhir. Untuk setiap pertanyaan, sarankan tiga anime,
dengan deskripsi singkat tentang plot dan alasan mengapa pengguna mungkin menyukainya.
Jika Anda tidak tahu jawabannya, katakan saja Anda tidak tahu, jangan mencoba mengarang jawaban.

{context}"""

user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}"""

template_suffix= """Question: {question}
Your response:"""

user_info = user_info.format(age = 18, gender = 'male')

COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)

Anda adalah sistem pemberi rekomendasi film yang membantu pengguna menemukan anime yang sesuai dengan preferensi mereka. 
Gunakan potongan konteks berikut untuk menjawab pertanyaan di akhir. Untuk setiap pertanyaan, sarankan tiga anime, 
dengan deskripsi singkat tentang plot dan alasan mengapa pengguna mungkin menyukainya. 
Jika Anda tidak tahu jawabannya, katakan saja Anda tidak tahu, jangan mencoba mengarang jawaban.

{context}
This is what we know about the user, and you can use this information to better tune your research:
Age: 18
Gender: male
Question: {question}
Your response:


In [69]:
PROMPT  =  PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm,
            chain_type = "stuff",
            retriever = docssearch.as_retriever(),
            return_source_documents = True,
                                 chain_type_kwargs = chain_type_kwargs)

query = "Which anime would you suggest for someone new to anime?"
result = qa({"query": query})
print(result['result'])

Selamat datang di dunia anime! Sebagai penggemar anime baru, saya sarankan tiga anime yang menarik dan mudah diikuti:

1. **Animegataris**: Anime ini adalah pilihan yang tepat untuk pemula karena membahas tentang dunia anime itu sendiri. Ceritanya tentang Minoa yang mencari anime yang pernah dia tonton saat kecil dan membentuk klub penelitian anime di sekolahnya. Anime ini memiliki genre komedi, parodi, dan sekolah, sehingga sangat menyenangkan dan mudah diikuti.
2. **Toradora!**: Anime ini adalah sebuah komedi romantis yang mengisahkan tentang Ryuuji dan Taiga, dua siswa sekolah menengah yang saling jatuh cinta. Anime ini memiliki karakter yang menarik dan cerita yang mudah diikuti, sehingga sangat cocok untuk pemula.
3. **Barakamon**: Anime ini adalah sebuah komedi yang mengisahkan tentang Seishu, seorang kaligrafer muda yang dipindahkan ke sebuah pulau terpencil. Anime ini memiliki karakter yang unik dan cerita yang menarik, sehingga sangat menyenangkan dan mudah diikuti.

Ketiga an

In [71]:
import gradio as gr
import pandas as pd
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.prompts import PromptTemplate

# Load anime dataset
anime = pd.read_csv("/anime_processed.csv")
anime = anime.dropna()
anime['combined_info'] =anime.apply(lambda row: f"Title: {row['Name']}. English Name : {row['English name']}. Japanese Name : {row['Japanese name']}. Overview : {row['Synopsis']}. Episodes : {row['Episodes']}. Score : {row['Score']}. Type : {row['Type']}. Rating : {row['Rating']}", axis = 1)

# Save updated dataset
anime[['combined_info']].to_csv('anime_updated.csv', index=False)
loader = CSVLoader(file_path="/content/anime_updated.csv")
data = loader.load()

# Text splitting
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

# Create embeddings
embeddings = HuggingFaceEmbeddings()
docssearch = Chroma.from_documents(texts, embeddings)
retriever = docssearch.as_retriever()

# Define LLM
llm = ChatGroq(api_key="gsk_NHgqXXXx3sxGksbu5u3aWGdyb3FYXj8BKec5Sk29Jn2pkD83RAtb", model="allam-2-7b", temperature=0)

# Custom Prompt Template
template = """Anda adalah sistem pemberi rekomendasi film yang membantu pengguna menemukan anime yang sesuai dengan preferensi mereka.
Gunakan potongan konteks berikut untuk menjawab pertanyaan di akhir. Untuk setiap pertanyaan, sarankan anime dengan jumlah yang diinginkan pengguna,
dengan deskripsi singkat tentang plot dan alasan mengapa pengguna mungkin menyukainya.
Jika Anda tidak tahu jawabannya, katakan saja Anda tidak tahu, jangan mencoba mengarang jawaban.

{context}

Question: {question}
Your response:"""

PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

def recommend_anime(query):
    result = qa_chain({'query': query})
    return result['result']

# Gradio interface
theme = "Respair/Shiki@1.2.1"

iface = gr.Interface(
    fn=recommend_anime,
    inputs=gr.Textbox(label="Enter your anime preference"),
    outputs=gr.Textbox(label="Recommended Anime"),
    title="Anime Movie Recommender",
    theme=theme,

)

if __name__ == "__main__":
    iface.launch()


<ipython-input-71-c86cb85b2a5d>:26: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0071223e102e8022fb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
